<a href="https://colab.research.google.com/github/jjlazo79/API-CFDB/blob/master/waybackmachine_by_equisele.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Credits to @equisele
# https://twitter.com/equisele/status/1640441625238183955
import os
import requests
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup
from google.colab import drive

In [ ]:
# Montamos Google Drive. Se solicitan permisos.
# Así se guardarán los html en una carpeta de tu cuenta.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Introduce un dominio que se encuentre en web.archive.org
domain = input("Introduce el dominio:")

Introduce el dominio:equisele.com


In [ ]:
# Creamos el nombre de la carpeta basada en el nombre de dominio
folder_name = domain.replace(".", "_")
# Creamos la ruta completa
folder_path = os.path.join('/content/drive/MyDrive/', folder_name)
# Iteramos para evitar sobreescribir una carpeta que exista con el mismo nombre
# Mientras exista el nombre de la carpeta, itera el bucle hasta que no exista
# y crear la carpeta en Google Drive
i = 1
while os.path.exists(folder_path):
    folder_name = f"{domain.replace('.', '_')}_{i}"
    folder_path = os.path.join('/content/drive/MyDrive/', folder_name)
    i += 1
os.makedirs(folder_path)

In [ ]:
# Define la URL base de la API CDX de Archive.org
base_url = "http://web.archive.org/cdx/search/cdx"

# Define los parámetros de la solicitud GET
params = {
    'url': f'{domain}/*', # Dominio y asterisco para buscar todas las URL
    'output': 'json', # Formato de salida de los resultados
    'fl': 'timestamp,mimetype,original', # Campos para incluir en los resultados
    'collapse': 'urlkey' # Agrupar lso snapshots por url y elimina así duplicados
}

# Crea una sesión de requests para reutilizar la misma conexión
session = requests.Session()

# Realiza la solicitud GET a la API de CDX de Archive.org
response = session.get(base_url, params=params)
response.raise_for_status()

# Obtiene la respuesta en formato JSON
response_json = response.json()

# Creamos la función que descargará el snapshot y almacenará en una carpeta local
def download_file(snapshot):
    
    timestamp, mime, original_url = snapshot[:3]

    if mime.startswith('text/html') and original_url.endswith('html'):

        # Muestra en pantalla información del snapshot a descargar
        # print(f'{timestamp} {original_url}')

        # Define la url del snapshot a descargar
        url = f'http://web.archive.org/web/{timestamp}id_/{original_url}'

        # Muestra en pantalla información sobre la página a descargar.
        print(f'Crawling: {url}')

        # Inicializamos el título de la página
        title = ''

        try:
            # Realiza la solicitud GET a la página del snapshot
            response = session.get(url, timeout=30) # Timeout a 30s. Si quieres puedes bajarlo o subirlo según tus necesidades
            response.raise_for_status()

            # Parsea el contenido html
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extrae el título
            title = soup.title.string.strip()

        except (requests.exceptions.Timeout, requests.exceptions.RequestException):
            pass

        # Define el nombre de archivo con el que guardaremos la página html
        filename = os.path.join(folder_path, f"{title.replace('/', '_')}.html")

        # Si existe el nombre de archivo, no lo descargamos de nuevo
        if os.path.isfile(filename):
            return

        # Guardamos el contenido html
        with open(filename, 'w') as f:
            f.write(response.text)

# Utilizamos ThreadPoolExecutor para descargar en paralelo
with ThreadPoolExecutor(max_workers=10) as executor:

    # Ejecuta la función download_file para cada snapshot encontrado
    executor.map(download_file, response_json[1:])  # Se omite el primer elemento de la lista (header)